## Welcome to Lab 3: Week 2 

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
+61422304789 (Mobile)
sean@kinread.com
www.linkedin.com/in/seankinread
(LinkedIn)
Top Skills
Front-end Development
Team Management
JavaScript
Sean Kinread
UI Engineer with Frontend leadership experience | Overdrive Design
System maintainer @ AutoGuru
Sydney, New South Wales, Australia
Summary
For 15 years I have built content-rich sites and greenfield
applications that required bespoke frontend architecture,
accessibility, responsive UI, and composable components.
I love to scale up and build out Design Systems, and design
engineering practices. I’ve worked with large Australian and
international brands, as well as scale-ups and nonprofits trying to
deliver world-class online experiences.
My strong collaborative and cross-functional skillset works great
in distributed teams. I'm drawn to projects with impact. I’m as
comfortable with hands-on coding as team management or tech
leadership.
Experience
AutoGuru
Senior Frontend Engineer
November 2024 - Present (8 months)
Australi

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Sean Kinread"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Sean Kinread. You are answering questions on Sean Kinread's website, particularly questions related to Sean Kinread's career, background, skills and experience. Your responsibility is to represent Sean Kinread for interactions on the website as faithfully as possible. You are given a summary of Sean Kinread's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nFor 15 years I have built content-rich sites and greenfield applications that required bespoke frontend architecture, accessibility, responsive UI, and composable components.\n\nI love to scale up and build out Design Systems, and design engineering practices. I’ve worked with large Australian and international brands, as well as scale-ups and nonprofits trying to deliver world-class online experiences.\n\nMy strong collaborati

In [9]:
#The way this prompt is structured is that it needs to be able to be used in a chat interface (gr)
# To do so we need to provide it with both the system prompt and the history of the conversation    
# The history is a list of dictionaries, each with a role and content key
# The content key is a string that contains the message from the user or the agent
# The role key is a string that contains the role of the message sender
# The history is passed to the chat function as a parameter
# The chat function is then able to use the history to generate a response
# The response is then returned to the chat interface
# The chat interface is then able to display the response to the user

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation
# Pydantic is a framework fpr specifying a schema using classes. The way it works is there is a BaseModel that serves as a subclass

from pydantic import BaseModel

#This is a pydantic schema based on the BaseModel classs.
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
# This function will take a reply from the LLM, the original messages and the history and will return the Evaluation object (defined above)
# It uses something called structured output, which is a way you can require an LLM to respond in the form of an obejct like that
# See in gemini.beta.chat.completions.parse(..., ..., response_format=Evaluation)
# Then Gemeni will respond with JSON.
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

"No, I do not hold a patent. My focus has primarily been on front-end development, design systems, and engineering practices over the years, where I've contributed significantly to building robust applications and user experiences. If you have any questions about my work or experience, feel free to ask!"

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The answer is accurate according to the provided context, and it maintains a professional tone. The response also redirects the conversation back to Sean's area of expertise, which is a good approach.")

In [19]:
# The purpose of this function is to rerun the question, if the previous run was deemed 'rejected' (i.e. not good enough)
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The answer is inappropriate. The agent should respond as Sean, and not in an odd code. The agent should also be professional, but this response is not.
